In [14]:
import pandas as pd

original_data = pd.read_csv('../data/_Process_monitoring__202303251137.csv', sep=',')

# eliminar columnas inutiles
original_data.drop(columns=['velocity'], inplace=True)

original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1685070 entries, 0 to 1685069
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   id          1685070 non-null  int64  
 1   date        1685070 non-null  object 
 2   time        1685070 non-null  object 
 3   lap         1685070 non-null  int64  
 4   latitude    1685070 non-null  float64
 5   longitude   1685070 non-null  float64
 6   navigation  1685070 non-null  int64  
 7   vehicle_id  1685070 non-null  object 
dtypes: float64(2), int64(3), object(3)
memory usage: 102.8+ MB


In [8]:
stops = pd.read_csv('../data/busstops_ida.csv', sep=',')
stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   number      84 non-null     int64  
 1   id          84 non-null     int64  
 2   name        84 non-null     object 
 3   navigation  84 non-null     int64  
 4   latitude    84 non-null     float64
 5   longitude   84 non-null     float64
 6   radio       84 non-null     int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 4.7+ KB


In [ ]:
# output: un dataset que contenga los puntos que pasaron por un stop